# **W1D1: Introduction to the Climate System and Xarray**

## Overview

The first three tutorials of today will introduce the basics of gridded, labeled data with Xarray. Since Xarray introduces additional abstractions on top of plain arrays of data, our goal is to show why these abstractions are useful and how they frequently lead to simpler, more robust code.

We'll cover these topics:

1. Create a `DataArray`, one of the core object types in Xarray
1. Understand how to use named coordinates and metadata in a `DataArray`
1. Combine individual `DataArrays` into a `Dataset`, the other core object type in Xarray
1. Subset, slice, and interpolate the data using named coordinates
1. Open netCDF data using XArray
1. Basic subsetting and aggregation of a `Dataset`
1. Brief introduction to plotting with Xarray

# **Tutorial 1: Creating DataArrays and Datasets to Assess Global Climate Data**


**Week 1, Day 1, Introduction to the Climate System**

**Content creators:** Sloane Garelick, Julia Kent

**Content reviewers:** Danika Gupta, Younkap Nina Duplex 

**Content editors:** Yosmely Bermúdez, Chi Zhang

**Production editors:** TBD

**Our 2023 Sponsors:** TBD

###**Code and Data Sources**

Code and data for this tutorial is based on existing content from [Project Pythia](https://foundations.projectpythia.org/core/xarray/xarray-intro.html).

## **Tutorial 1 Objectives**


As you just learned in the Introduction to Climate video, variations in global climate involve various forcings, feedbacks, and interactions between multiple processes and systems. Because of this complexity, global climate datasets are often very large with multiple dimensions and variables.

One useful computational tool for organizing, analyzing and interpreting large global datasets is XArray, an open source project and Python package that makes working with labelled multi-dimensional arrays simple and efficient.

In this first tutorial, we will use the `DataArray` and `Dataset` objects, which are used to represent and manipulate spatial data, to practice organizing large global climate datasets and to understand variations in Earth's climate system.

## Imports

Simmilar to `numpy`, `np`; `pandas`, `pd`; you may often encounter `xarray` imported within a shortened namespace as `xr`. `pythia_datasets` provides example data for us to work with.

In [1]:
!pip install datetime

!pip install numpy
!pip install pandas
!pip install xarray
!pip install pythia_datasets

In [2]:
from datetime import timedelta

import numpy as np
import pandas as pd
import xarray as xr
from pythia_datasets import DATASETS

## Introducing the `DataArray` and `Dataset`

Xarray expands on the capabilities on NumPy arrays, providing a lot of streamlined data manipulation. It is similar in that respect to Pandas, but whereas Pandas excels at working with tabular data, Xarray is focused on N-dimensional arrays of data (i.e. grids). Its interface is based largely on the netCDF data model (variables, attributes, and dimensions), but it goes beyond the traditional netCDF interfaces to provide functionality similar to netCDF-java's [Common Data Model (CDM)](https://docs.unidata.ucar.edu/netcdf-java/current/userguide/common_data_model_overview.html). 

### Creation of a `DataArray` object

The `DataArray` is one of the basic building blocks of Xarray (see docs [here](http://xarray.pydata.org/en/stable/user-guide/data-structures.html#dataarray)). It provides a `numpy.ndarray`-like object that expands to provide two critical pieces of functionality:

1. Coordinate names and values are stored with the data, making slicing and indexing much more powerful
2. It has a built-in container for attributes

Here we'll initialize a `DataArray` object by wrapping a plain NumPy array, and explore a few of its properties.

#### Generate a random numpy array

For our first example, we'll just create a random array of "temperature" data in units of Kelvin:

In [3]:
data = 283 + 5 * np.random.randn(5, 3, 4)
data

array([[[290.38594232, 284.24338095, 282.35178984, 290.37825651],
        [286.08536722, 276.74023865, 275.28380699, 288.50789742],
        [290.65107129, 273.81673023, 284.98879086, 279.45012141]],

       [[275.90493357, 292.1659756 , 284.32054296, 287.67712745],
        [276.51685132, 276.53835269, 280.16213972, 293.98934256],
        [287.28623689, 279.5904951 , 275.51828994, 284.85656559]],

       [[281.64782943, 292.63221553, 272.224315  , 283.79925383],
        [288.33074219, 283.80365394, 278.72016626, 287.4613854 ],
        [288.78868923, 286.85602622, 276.57392588, 285.97080864]],

       [[287.71745686, 282.22776801, 277.94686845, 284.93149128],
        [285.65778418, 280.34325741, 279.25414918, 283.26866583],
        [283.64881322, 281.61884077, 275.11212708, 280.62696924]],

       [[285.50208437, 277.65232834, 285.97681329, 279.03257875],
        [280.76823111, 275.05316025, 281.40251272, 284.64375484],
        [278.03907773, 280.99275816, 274.14410447, 280.57060285]]])

#### Wrap the array: first attempt

Now we create a basic `DataArray` just by passing our plain `data` as input:

In [4]:
temp = xr.DataArray(data)
temp

<xarray.DataArray (dim_0: 5, dim_1: 3, dim_2: 4)>
array([[[290.38594232, 284.24338095, 282.35178984, 290.37825651],
        [286.08536722, 276.74023865, 275.28380699, 288.50789742],
        [290.65107129, 273.81673023, 284.98879086, 279.45012141]],

       [[275.90493357, 292.1659756 , 284.32054296, 287.67712745],
        [276.51685132, 276.53835269, 280.16213972, 293.98934256],
        [287.28623689, 279.5904951 , 275.51828994, 284.85656559]],

       [[281.64782943, 292.63221553, 272.224315  , 283.79925383],
        [288.33074219, 283.80365394, 278.72016626, 287.4613854 ],
        [288.78868923, 286.85602622, 276.57392588, 285.97080864]],

       [[287.71745686, 282.22776801, 277.94686845, 284.93149128],
        [285.65778418, 280.34325741, 279.25414918, 283.26866583],
        [283.64881322, 281.61884077, 275.11212708, 280.62696924]],

       [[285.50208437, 277.65232834, 285.97681329, 279.03257875],
        [280.76823111, 275.05316025, 281.40251272, 284.64375484],
        [278.03907773, 280.99275816, 274.14410447, 280.57060285]]])
Dimensions without coordinates: dim_0, dim_1, dim_2

Note two things:

1. Xarray generates some basic dimension names for us (`dim_0`, `dim_1`, `dim_2`). We'll improve this with better names in the next example.
2. Wrapping the numpy array in a `DataArray` gives us a rich display in the notebook! (Try clicking the array symbol to expand or collapse the view)

#### Assign dimension names

Much of the power of Xarray comes from making use of named dimensions. So let's add some more useful names! We can do that by passing an ordered list of names using the keyword argument `dims`:

In [5]:
temp = xr.DataArray(data, dims=['time', 'lat', 'lon'])
temp

<xarray.DataArray (time: 5, lat: 3, lon: 4)>
array([[[290.38594232, 284.24338095, 282.35178984, 290.37825651],
        [286.08536722, 276.74023865, 275.28380699, 288.50789742],
        [290.65107129, 273.81673023, 284.98879086, 279.45012141]],

       [[275.90493357, 292.1659756 , 284.32054296, 287.67712745],
        [276.51685132, 276.53835269, 280.16213972, 293.98934256],
        [287.28623689, 279.5904951 , 275.51828994, 284.85656559]],

       [[281.64782943, 292.63221553, 272.224315  , 283.79925383],
        [288.33074219, 283.80365394, 278.72016626, 287.4613854 ],
        [288.78868923, 286.85602622, 276.57392588, 285.97080864]],

       [[287.71745686, 282.22776801, 277.94686845, 284.93149128],
        [285.65778418, 280.34325741, 279.25414918, 283.26866583],
        [283.64881322, 281.61884077, 275.11212708, 280.62696924]],

       [[285.50208437, 277.65232834, 285.97681329, 279.03257875],
        [280.76823111, 275.05316025, 281.40251272, 284.64375484],
        [278.03907773, 280.99275816, 274.14410447, 280.57060285]]])
Dimensions without coordinates: time, lat, lon

This is already improved upon from a NumPy array, because we have names for each of the dimensions (or axes in NumPy parlance). Even better, we can take arrays representing the values for the coordinates for each of these dimensions and associate them with the data when we create the `DataArray`. We'll see this in the next example.

### Create a `DataArray` with named Coordinates

#### Make time and space coordinates

Here we will use [Pandas](../pandas) to create an array of [datetime data](../datetime), which we will then use to create a `DataArray` with a named coordinate `time`.

In [6]:
times = pd.date_range('2018-01-01', periods=5)
times

DatetimeIndex(['2018-01-01', '2018-01-02', '2018-01-03', '2018-01-04',
               '2018-01-05'],
              dtype='datetime64[ns]', freq='D')

We'll also create arrays to represent sample longitude and latitude:

In [7]:
lons = np.linspace(-120, -60, 4)
lats = np.linspace(25, 55, 3)

#### Initialize the `DataArray` with complete coordinate info

When we create the `DataArray` instance, we pass in the arrays we just created:

In [8]:
temp = xr.DataArray(data, coords=[times, lats, lons], dims=['time', 'lat', 'lon'])
temp

<xarray.DataArray (time: 5, lat: 3, lon: 4)>
array([[[290.38594232, 284.24338095, 282.35178984, 290.37825651],
        [286.08536722, 276.74023865, 275.28380699, 288.50789742],
        [290.65107129, 273.81673023, 284.98879086, 279.45012141]],

       [[275.90493357, 292.1659756 , 284.32054296, 287.67712745],
        [276.51685132, 276.53835269, 280.16213972, 293.98934256],
        [287.28623689, 279.5904951 , 275.51828994, 284.85656559]],

       [[281.64782943, 292.63221553, 272.224315  , 283.79925383],
        [288.33074219, 283.80365394, 278.72016626, 287.4613854 ],
        [288.78868923, 286.85602622, 276.57392588, 285.97080864]],

       [[287.71745686, 282.22776801, 277.94686845, 284.93149128],
        [285.65778418, 280.34325741, 279.25414918, 283.26866583],
        [283.64881322, 281.61884077, 275.11212708, 280.62696924]],

       [[285.50208437, 277.65232834, 285.97681329, 279.03257875],
        [280.76823111, 275.05316025, 281.40251272, 284.64375484],
        [278.03907773, 280.99275816, 274.14410447, 280.57060285]]])
Coordinates:
  * time     (time) datetime64[ns] 2018-01-01 2018-01-02 ... 2018-01-05
  * lat      (lat) float64 25.0 40.0 55.0
  * lon      (lon) float64 -120.0 -100.0 -80.0 -60.0

#### Set useful attributes

...and while we're at it, we can also set some attribute metadata:

In [9]:
temp.attrs['units'] = 'kelvin'
temp.attrs['standard_name'] = 'air_temperature'

temp

<xarray.DataArray (time: 5, lat: 3, lon: 4)>
array([[[290.38594232, 284.24338095, 282.35178984, 290.37825651],
        [286.08536722, 276.74023865, 275.28380699, 288.50789742],
        [290.65107129, 273.81673023, 284.98879086, 279.45012141]],

       [[275.90493357, 292.1659756 , 284.32054296, 287.67712745],
        [276.51685132, 276.53835269, 280.16213972, 293.98934256],
        [287.28623689, 279.5904951 , 275.51828994, 284.85656559]],

       [[281.64782943, 292.63221553, 272.224315  , 283.79925383],
        [288.33074219, 283.80365394, 278.72016626, 287.4613854 ],
        [288.78868923, 286.85602622, 276.57392588, 285.97080864]],

       [[287.71745686, 282.22776801, 277.94686845, 284.93149128],
        [285.65778418, 280.34325741, 279.25414918, 283.26866583],
        [283.64881322, 281.61884077, 275.11212708, 280.62696924]],

       [[285.50208437, 277.65232834, 285.97681329, 279.03257875],
        [280.76823111, 275.05316025, 281.40251272, 284.64375484],
        [278.03907773, 280.99275816, 274.14410447, 280.57060285]]])
Coordinates:
  * time     (time) datetime64[ns] 2018-01-01 2018-01-02 ... 2018-01-05
  * lat      (lat) float64 25.0 40.0 55.0
  * lon      (lon) float64 -120.0 -100.0 -80.0 -60.0
Attributes:
    units:          kelvin
    standard_name:  air_temperature

#### Attributes are not preserved by default!

Notice what happens if we perform a mathematical operaton with the `DataArray`: the coordinate values persist, but the attributes are lost. This is done because it is very challenging to know if the attribute metadata is still correct or appropriate after arbitrary arithmetic operations.

To illustrate this, we'll do a simple unit conversion from Kelvin to Celsius:

In [10]:
temp_in_celsius = temp - 273.15
temp_in_celsius

<xarray.DataArray (time: 5, lat: 3, lon: 4)>
array([[[17.23594232, 11.09338095,  9.20178984, 17.22825651],
        [12.93536722,  3.59023865,  2.13380699, 15.35789742],
        [17.50107129,  0.66673023, 11.83879086,  6.30012141]],

       [[ 2.75493357, 19.0159756 , 11.17054296, 14.52712745],
        [ 3.36685132,  3.38835269,  7.01213972, 20.83934256],
        [14.13623689,  6.4404951 ,  2.36828994, 11.70656559]],

       [[ 8.49782943, 19.48221553, -0.925685  , 10.64925383],
        [15.18074219, 10.65365394,  5.57016626, 14.3113854 ],
        [15.63868923, 13.70602622,  3.42392588, 12.82080864]],

       [[14.56745686,  9.07776801,  4.79686845, 11.78149128],
        [12.50778418,  7.19325741,  6.10414918, 10.11866583],
        [10.49881322,  8.46884077,  1.96212708,  7.47696924]],

       [[12.35208437,  4.50232834, 12.82681329,  5.88257875],
        [ 7.61823111,  1.90316025,  8.25251272, 11.49375484],
        [ 4.88907773,  7.84275816,  0.99410447,  7.42060285]]])
Coordinates:
  * time     (time) datetime64[ns] 2018-01-01 2018-01-02 ... 2018-01-05
  * lat      (lat) float64 25.0 40.0 55.0
  * lon      (lon) float64 -120.0 -100.0 -80.0 -60.0

For an in-depth discussion of how Xarray handles metadata, start in the Xarray docs [here](http://xarray.pydata.org/en/stable/getting-started-guide/faq.html#approach-to-metadata).

### The `Dataset`: a container for `DataArray`s with shared coordinates

Along with `DataArray`, the other key object type in Xarray is the `Dataset`: a dictionary-like container that holds one or more `DataArray`s, which can also optionally share coordinates (see docs [here](http://xarray.pydata.org/en/stable/user-guide/data-structures.html#dataset)).

The most common way to create a `Dataset` object is to load data from a file (see [below](#Opening-netCDF-data)). Here, instead, we will create another `DataArray` and combine it with our `temp` data.

This will illustrate how the information about common coordinate axes is used.

#### Create a pressure `DataArray` using the same coordinates

This code mirrors how we created the `temp` object above.

In [11]:
pressure_data = 1000.0 + 5 * np.random.randn(5, 3, 4)
pressure = xr.DataArray(
    pressure_data, coords=[times, lats, lons], dims=['time', 'lat', 'lon']
)
pressure.attrs['units'] = 'hPa'
pressure.attrs['standard_name'] = 'air_pressure'

pressure

<xarray.DataArray (time: 5, lat: 3, lon: 4)>
array([[[ 996.30102459,  999.4795644 ,  997.12406028, 1002.85558118],
        [ 994.87363982, 1003.26505552,  992.75831145,  986.86770094],
        [1002.2247732 ,  998.72598719,  999.10564335,  995.66158155]],

       [[ 991.27629283, 1007.40035441,  997.66894789, 1000.25625323],
        [ 997.95029584, 1007.50423178, 1001.99839482,  995.90754689],
        [1001.29163045,  991.50490653,  999.6794448 , 1006.36018554]],

       [[ 996.5739886 ,  997.53160095, 1012.74276442,  995.68844442],
        [ 990.67935419,  997.74723175,  996.55424976,  993.60294526],
        [1000.23861706, 1004.54624809,  998.34701556,  999.89679839]],

       [[ 998.8543107 ,  996.0305689 ,  997.94413813, 1002.85643227],
        [ 998.36568254,  991.57436998,  997.34783763, 1005.49314066],
        [1002.3083235 ,  998.46461289,  996.56425514,  998.841328  ]],

       [[ 999.2340605 ,  998.19505919,  997.2198862 ,  997.27255225],
        [ 994.9321904 ,  997.77142711, 1001.58669041, 1004.76585218],
        [ 997.31082228,  998.873691  ,  992.2409281 ,  997.41422855]]])
Coordinates:
  * time     (time) datetime64[ns] 2018-01-01 2018-01-02 ... 2018-01-05
  * lat      (lat) float64 25.0 40.0 55.0
  * lon      (lon) float64 -120.0 -100.0 -80.0 -60.0
Attributes:
    units:          hPa
    standard_name:  air_pressure

#### Create a `Dataset` object

Each `DataArray` in our `Dataset` needs a name! 

The most straightforward way to create a `Dataset` with our `temp` and `pressure` arrays is to pass a dictionary using the keyword argument `data_vars`:

In [12]:
ds = xr.Dataset(data_vars={'Temperature': temp, 'Pressure': pressure})
ds

<xarray.Dataset>
Dimensions:      (time: 5, lat: 3, lon: 4)
Coordinates:
  * time         (time) datetime64[ns] 2018-01-01 2018-01-02 ... 2018-01-05
  * lat          (lat) float64 25.0 40.0 55.0
  * lon          (lon) float64 -120.0 -100.0 -80.0 -60.0
Data variables:
    Temperature  (time, lat, lon) float64 290.4 284.2 282.4 ... 274.1 280.6
    Pressure     (time, lat, lon) float64 996.3 999.5 997.1 ... 992.2 997.4

Notice that the `Dataset` object `ds` is aware that both data arrays sit on the same coordinate axes.

#### Access Data variables and Coordinates in a `Dataset`

We can pull out any of the individual `DataArray` objects in a few different ways.

Using the "dot" notation:

In [13]:
ds.Pressure

<xarray.DataArray 'Pressure' (time: 5, lat: 3, lon: 4)>
array([[[ 996.30102459,  999.4795644 ,  997.12406028, 1002.85558118],
        [ 994.87363982, 1003.26505552,  992.75831145,  986.86770094],
        [1002.2247732 ,  998.72598719,  999.10564335,  995.66158155]],

       [[ 991.27629283, 1007.40035441,  997.66894789, 1000.25625323],
        [ 997.95029584, 1007.50423178, 1001.99839482,  995.90754689],
        [1001.29163045,  991.50490653,  999.6794448 , 1006.36018554]],

       [[ 996.5739886 ,  997.53160095, 1012.74276442,  995.68844442],
        [ 990.67935419,  997.74723175,  996.55424976,  993.60294526],
        [1000.23861706, 1004.54624809,  998.34701556,  999.89679839]],

       [[ 998.8543107 ,  996.0305689 ,  997.94413813, 1002.85643227],
        [ 998.36568254,  991.57436998,  997.34783763, 1005.49314066],
        [1002.3083235 ,  998.46461289,  996.56425514,  998.841328  ]],

       [[ 999.2340605 ,  998.19505919,  997.2198862 ,  997.27255225],
        [ 994.9321904 ,  997.77142711, 1001.58669041, 1004.76585218],
        [ 997.31082228,  998.873691  ,  992.2409281 ,  997.41422855]]])
Coordinates:
  * time     (time) datetime64[ns] 2018-01-01 2018-01-02 ... 2018-01-05
  * lat      (lat) float64 25.0 40.0 55.0
  * lon      (lon) float64 -120.0 -100.0 -80.0 -60.0
Attributes:
    units:          hPa
    standard_name:  air_pressure

... or using dictionary access like this:

In [14]:
ds['Pressure']

<xarray.DataArray 'Pressure' (time: 5, lat: 3, lon: 4)>
array([[[ 996.30102459,  999.4795644 ,  997.12406028, 1002.85558118],
        [ 994.87363982, 1003.26505552,  992.75831145,  986.86770094],
        [1002.2247732 ,  998.72598719,  999.10564335,  995.66158155]],

       [[ 991.27629283, 1007.40035441,  997.66894789, 1000.25625323],
        [ 997.95029584, 1007.50423178, 1001.99839482,  995.90754689],
        [1001.29163045,  991.50490653,  999.6794448 , 1006.36018554]],

       [[ 996.5739886 ,  997.53160095, 1012.74276442,  995.68844442],
        [ 990.67935419,  997.74723175,  996.55424976,  993.60294526],
        [1000.23861706, 1004.54624809,  998.34701556,  999.89679839]],

       [[ 998.8543107 ,  996.0305689 ,  997.94413813, 1002.85643227],
        [ 998.36568254,  991.57436998,  997.34783763, 1005.49314066],
        [1002.3083235 ,  998.46461289,  996.56425514,  998.841328  ]],

       [[ 999.2340605 ,  998.19505919,  997.2198862 ,  997.27255225],
        [ 994.9321904 ,  997.77142711, 1001.58669041, 1004.76585218],
        [ 997.31082228,  998.873691  ,  992.2409281 ,  997.41422855]]])
Coordinates:
  * time     (time) datetime64[ns] 2018-01-01 2018-01-02 ... 2018-01-05
  * lat      (lat) float64 25.0 40.0 55.0
  * lon      (lon) float64 -120.0 -100.0 -80.0 -60.0
Attributes:
    units:          hPa
    standard_name:  air_pressure

We'll return to the `Dataset` object when we start loading data from files.